# TP2 : Extraction d'informations

Sur la base des éléments méthodologiques et des enseignements techniques présentés lors du cours théorique, il est demandé dans le cadre de ce TP :
- de se familiariser avec les techniques d’enrichissement (extraction de mots-clés, reconnaissance d’entités nommées, analyse de sentiment...)
- d’appliquer celles-ci au corpus de bulletins de la ville de Bruxelles préalablement convertis au format .txt

Pour ce faire, vous utiliserez différentes librairies Python vues au cours.

## 1. Keywords

### Librairies utilisées dans le code

In [1]:
import os
from collections import defaultdict, Counter
import textract
import nltk
from nltk.corpus import stopwords
import yake
from wordcloud import WordCloud
from IPython.display import Image

### Sélection des fichiers à analyser

#### Stocker la liste des fichiers contenus dans le dossier 'txt'

In [2]:
txt_path = "../data/txt/"
files = sorted(os.listdir(txt_path)) # stocker dans 'files' la liste des fichiers contenus dans le dossier 'txt'
len(files)

2827

#### Lister les fichiers pour Bruxelles et pour Laeken

In [3]:
bxl_1930 = [f for f in files if f.startswith('Bxl_1930')]
print(len(bxl_1930))

lkn_1930 = [f for f in files if f.startswith('Lkn_1930')]
print(len(lkn_1930))

31
0


#### Vérification du nombre de fichiers

In [4]:
count_years = defaultdict(int)

for f in files: 
    if "_" in f and f.endswith("txt"): 
        elems = f.split("_") 
        year = elems[1] 
        count_years[year] += 1 
        
n_1930 = count_years['1930']
print(n_1930)

31


### Récupérer le texte des fichiers

#### Créer le dossier adéquat

In [5]:
path_1930 = '../data/1930'
if not os.path.exists(path_1930):
    os.mkdir(path_1930)

#### Copier les fichiers désirés dans le dossier

In [6]:
for txt in bxl_1930:
    text = textract.process(os.path.join(txt_path, txt))
    with open(os.path.join(path_1930, txt), 'wb') as f: 
        f.write(text) 

#### Concaténer le texte des fichiers

In [7]:
!cat ../data/1930/*.txt > ../data/1930.txt

#### Compter le nombre de mots repris dans le fichier

In [8]:
!wc -w ../data/1930.txt

968678 ../data/1930.txt


In [9]:
texts_1930 = open(os.path.join(f"../data/", "1930.txt"), 'r').read()
texts_1930[:500]

'VILLE\n\nDE\n\nBRUXELLES\n\nBULLETIN COMMUNAL\nA N N É E\n\n1930\n\nTOME\n\nII\n\nBRUXELLES\nIMPRIMERIE\n\nE . G U Y O T , S. A.\n\n12. rue Pachéco, 12\n1 930\n\n\x0c\x0cN" l .\n\nCOMTTE RENDU DE LA SÉANCE DU 7 JUILLET IfWO.\n\nV I L L E\n\nD E\n\nBULLETIN\nAnnée\n\nC O N S E I L\nSéance\n\nB R U X E L L E S\n\nGO M M U N AI\n1930.\n\nC O M M U N A L\n\ndu 7 Juillet\n\n1930.\n\nPrésidence de M . ADOLPHE M A X , Bourgmestre.\n\nSOMMAIRE :\nPAGES.\n\n1.\n2.\n3.\n\nCommunications\n. . .\n5\nDémission de M. l\'Echevin Jacqmain\n20\nElection et prestation de serment d'

### Extraction des mots clés

#### Instancier l'extracteur de mots clés

In [10]:
kw_extractor = yake.KeywordExtractor(lan="fr", top=50)
kw_extractor

In [11]:
keywords = kw_extractor.extract_keywords(texts_1930.lower()) 
keywords

[('rue', 2.249628377502558e-05),
 ("commission d'assistance publique", 2.587782417837962e-05),
 ('francs', 2.855334223198418e-05),
 ('conseil communal', 3.79514469207672e-05),
 ("vote d'un crédit", 5.589934068841265e-05),
 ('service', 5.9490167802554696e-05),
 ('ville', 6.526590109041209e-05),
 ('francs francs francs', 7.282438793271651e-05),
 ('bruxelles', 7.602535110817505e-05),
 ('frais', 9.607753438571384e-05),
 ('avis favorable', 9.948702081089956e-05),
 ("d'assistance publique", 0.00010413718787898636),
 ('mesdames et messieurs', 0.0001092176767512546),
 ('recettes', 0.00011519499322407083),
 ('conseil', 0.0001213116577344998),
 ("d'une", 0.0001327705744370901),
 ('travaux', 0.00013898747581888842),
 ('cours', 0.0001408300904078015),
 ('publique', 0.00014338484881775332),
 ('écoles primaires', 0.00015342783245787857),
 ('collège', 0.0001543631605944976),
 ('maison rue', 0.00015581086890629174),
 ('bourgmestre', 0.00016293115515389296),
 ('communal', 0.00016753980144313058),
 ("d'

In [12]:
ignored = ["conseil communal", "conseil général", "bruxelles"]
ignored

['conseil communal', 'conseil général', 'bruxelles']

#### Ne garder que les bigrammes

In [13]:
sw1 = []
for kw, score in keywords:
    words = kw.split()
    if len(words) == 1 and kw.lower() not in ignored:
        sw1.append(kw)
ignored2 = sw1 + ignored
print(sorted(ignored2))

['bourgmestre', 'bruxelles', 'budget', "c'est", 'collège', 'communal', 'conseil', 'conseil communal', 'conseil général', 'cours', 'crédit', "d'un", "d'une", 'dépenses', 'faire', 'frais', 'francs', "l'echevin", 'mais', 'membres', 'personnel', 'publique', 'question', 'recettes', 'rue', 'rues', 'service', 'services', 'total', 'travaux', 'van', 'ville', 'écoles']


#### Ajouter les mongrammes à la liste des stopwords

#### Analyse séquentielle en fonction des tomes

In [14]:
for f in sorted(bxl_1930):
    text = open(os.path.join(path_1930, f), 'r').read()
    keywords = kw_extractor.extract_keywords(text.lower()) # rajouter le lower 
    kept = []
    for kw, score in keywords:
        words = kw.split()
        if len(words) == 2 and kw.lower() not in ignored:
            kept.append(kw)
    print(f"{f} mentions these keywords: {', '.join(kept)}...\n")

Bxl_1930_Tome_II1_Part_1.txt mentions these keywords: crédits supplémentaires, avis favorable, d'un crédit, d'assistance publique, vote d'un, jeunes filles, crédits extraordinaires, commission d'assistance, francs francs, crédit communal...

Bxl_1930_Tome_II1_Part_10.txt mentions these keywords: affaire pendante, affaire terminée, acte administratif, d'assistance publique, cour d'appel, commission d'assistance, service d'assistance, ville d'un, société anonyme...

Bxl_1930_Tome_II1_Part_11.txt mentions these keywords: voir ecoles, voir art, voir affichage, affaire pendante, guerre afférents, voir enseignement, service médical, voir laboratoire, voir cours...

Bxl_1930_Tome_II1_Part_2.txt mentions these keywords: pensions communales, frais d'administration, l'école professionnelle, depense dépenses, frais généraux, écoles primaires, ecole normale, personnel frais, recettes ordinaires, reçu recettes...

Bxl_1930_Tome_II1_Part_3.txt mentions these keywords: idem idem, service extraordinai

### Nettoyer le fichier

#### Tokenisation

In [ ]:
words = nltk.wordpunct_tokenize(texts_1930)
print(f"{len(words)} words found")

print(words[:200])

#### Création de la 'stopwrods' de base enrichie

In [ ]:
sw = stopwords.words("french")
sw += ["les", "plus", "cette", "fait", "faire", "être", "deux", "comme", "dont", "tout", 
       "ils", "bien", "sans", "peut", "tous", "après", "ainsi", "donc", "cet", "sous",
       "celle", "entre", "encore", "toutes", "pendant", "moins", "dire", "cela", "non",
       "faut", "trois", "aussi", "dit", "avoir", "doit", "contre", "depuis", "autres",
       "van", "het", "autre", "jusqu"]

#### Calculer la taille du vocabulaire

In [ ]:
kept = []

for w in words:
    if len(w) > 2 and w.isalpha() and w.lower() not in sw:
        kept.append(w.lower())
        
voc = set(kept)

print(f"{len(kept)} words kept ({len(voc)} different word forms)")
print(kept[:200])

OU

In [ ]:
kept = [w.lower() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
voc = set(kept)
print(f"{len(kept)} words kept ({len(voc)} different word forms)")

#### Récupérer les mots les plus fréquents

In [ ]:
fdist = nltk.FreqDist(kept)
fdist.most_common(10)

#### Ajout d'une liste de mots à ignorer

In [ ]:
sw += ["rue", "francs", "ville", "bruxelles", "conseil"] 

print(sorted(sw))

In [ ]:
kept = [w.lower() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
voc = set(kept)
print(f"{len(kept)} words kept ({len(voc)} different word forms)")

In [ ]:
fdist = nltk.FreqDist(kept)
fdist.most_common(10)

In [ ]:
sw += ["collège", "communal", "publique", "bourgmestre"] 

print(sorted(sw))

In [ ]:
kept = [w.lower() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
voc = set(kept)
print(f"{len(kept)} words kept ({len(voc)} different word forms)")

In [ ]:
fdist = nltk.FreqDist(kept)
fdist.most_common(20)

In [ ]:
sw += ["total", "echevin", "membres", "divers", "année", "administration", "article", "question", "diverses", "messieurs", "art"] 

print(sorted(sw))

### Enrichissement de la liste des stopwords

In [ ]:
print(keywords)

## 2. Named Entity Recognition